In [1]:
# TO DO: Create a python function to scrape the website for data
# 1. Scrape the dropdown selection menu for website urls containing html data tables

from gazpacho import get, Soup # Import gazpacho

url = "https://tax.alaska.gov/programs/oil/production/ans.aspx?" # Set the primary url value as a "string"

html = get(url) # Get the html from the url
soup = Soup(html) # Pass html into soup
list = soup.find('option')[1:] # Find all the 'option' html tags and add to list
length = len(list) # Number of records in the list (month/year)
urls = [] # Create an empty list for all the urls

for i in range(length): # Loop through all the html tags for 'options' for the length of the list
    options = list[i].attrs['value']
#     print(url + options)
    urls.append(url + options) # and add the value to the urls list
    
url_count = len(urls) # Number of urls in the list

for url in urls:
    scraped_df = pd.read_html(url)[6] # Read the 7th data table from each url
    scraped_df = pd.DataFrame(scraped_df) # Set scraped_df as a dataframe
    scraped_df = scraped_df.dropna(axis=1, how='all')
    scraped_df = scraped_df.dropna(axis=0, how='all')
    header_row = 1 # Identify row with header values
    scraped_df.columns = scraped_df.iloc[header_row] # Rename columns equal to row with header values
    scraped_df['Date'] = pd.to_datetime(scraped_df['Date'], errors='coerce') # Convert the 'Date' column to datetime datatype
    scraped_df = scraped_df.dropna(axis=0, how='any', subset=['Date']) # Drop any rows with NaN in 'Date'
    scraped_df = scraped_df.dropna(axis=1, how='all') # Drop columns that are all NaN
    cols = scraped_df.columns.drop('Date') # Create a variable for all columns except 'Date'
    scraped_df[cols] = scraped_df[cols].apply(pd.to_numeric, errors='coerce') # Converted all the columns to numeric
    file_df = pd.read_csv('Resources/alaska_oil_production.csv') # Read previously scraped data file into Pandas
    file_df['Date'] = pd.to_datetime(file_df['Date'], errors='coerce')
    merged_df = file_df.append(scraped_df) # Merge the scraped_df to the file_df
    merged_df = merged_df.sort_values(by='Date', ascending=False) # Sort by 'Date' in descending order
    merged_df['duplicates'] = merged_df.duplicated('Date', keep='last') # Create a column to assign duplicates T/F
    merged_df = merged_df[merged_df['duplicates'] == False] # Save df with duplicates equal to False only
    merged_df = merged_df.drop(columns=['duplicates'])# Delete the 'duplicates' column if dupe_count = 0
    merged_df = merged_df.reset_index(drop=True) # Reset the merged dataframe index
    merged_df.to_csv('Resources/alaska_oil_production.csv', mode='w', index=False) # Write merged dataframe to existing csv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PermissionError: [Errno 13] Permission denied: 'Resources/alaska_oil_production.csv'

In [2]:
merged_df

,Date,Prudhoe Bay,Kuparuk,Endicott,Lisburne,Alpine,ANS,Inventories,Milne Point,Northstar,Cook Inlet,Alaska,PS# Temperature
0,2021-11-23,331815.0,106591.0,0.0,21516.0,46616.0,506538.0,2842017.0,NaN,NaN,NaN,NaN,NaN
1,2021-11-22,328363.0,112695.0,0.0,21344.0,47018.0,509420.0,2339727.0,NaN,NaN,NaN,NaN,NaN
2,2021-11-21,324463.0,106084.0,0.0,20676.0,48046.0,499269.0,3676840.0,NaN,NaN,NaN,NaN,NaN
3,2021-11-20,325643.0,106885.0,0.0,20856.0,46364.0,499749.0,4445129.0,NaN,NaN,NaN,NaN,NaN
4,2021-11-19,328818.0,106872.0,0.0,20814.0,48095.0,504598.0,4143875.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7618,2001-01-05,577437.0,235070.0,37116.0,87969.0,49313.0,1038011.0,5477997.0,51106.0,NaN,28361.0,1066372.0,-25.5
7619,2001-01-04,552823.0,232217.0,51302.0,37424.0,88434.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7620,2001-01-03,577810.0,231431.0,51236.0,38211.0,88144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7621,2001-01-02,585088.0,233745.0,38762.0,85462.0,72783.0,1067766.0,5653902.0,51926.0,NaN,28467.0,1096233.0,-17.6


In [ ]:
# file_df = pd.read_csv('Resources/alaska_oil_production.csv') # Read previously scraped data file into Pandas
# file_df['Date'] = pd.to_datetime(file_df['Date'], errors='coerce')
# merged_df = file_df.append(scraped_df) # Merge the scraped_df to the file_df
# merged_df = merged_df.sort_values(by='Date', ascending=False) # Sort by 'Date' in descending order
# merged_df['duplicates'] = merged_df.duplicated('Date', keep='last') # Create a column to assign duplicates T/F
# merged_df = merged_df[merged_df['duplicates'] == False] # Save df with duplicates equal to False only
# merged_df = merged_df.drop(columns=['duplicates'])# Delete the 'duplicates' column if dupe_count = 0
# merged_df = merged_df.reset_index(drop=True) # Reset the merged dataframe index
# merged_df.to_csv('Resources/alaska_oil_production.csv', mode='w', index=False) # Write merged dataframe to existing csv

In [ ]:
# merged_df

In [ ]:
# import pandas as pd
# first_url = urls[0] # first_url for standalone, use different code with for loop
# scraped_df = pd.read_html(first_url)[6] # Read the 7th data table from the firsts/each url
# scraped_df = pd.DataFrame(scraped_df) # Set scraped_df as a dataframe
# # scraped_df #show the last url scraped